# Pararelização do Índice de Dunn


## OpenMP

In [1]:
%%writefile dunn_openmp.c
/* dunn_openmp.c
 *
 * Portagem do script Python para C com OpenMP:
 * - Leitura de “cleaned_retail.csv”
 * - Cálculo de RFM (Recency, Frequency, Monetary)
 * - Padronização (StandardScaler)
 * - KMeans para k = 4…10
 * - Cálculo do Índice de Dunn (serial e paralelo)
 *
 * Para compilar: gcc -O2 -fopenmp -o dunn_openmp dunn_openmp.c -lm
 */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>
#include <omp.h>

/* CONFIGURAÇÕES GLOBAIS */
#define MAX_LINE_LEN   4096
#define MAX_CUSTOMERS  5000   /* tamanho máximo esperado de clientes únicos */
#define MAX_ROWS       300000   /* número máximo de linhas no CSV */
#define N_MAX          300000    /* limite de RFM após amostragem */
#define MAX_KMEANS_IT  100       /* número máximo de iterações no KMeans */
#define SEED           42

/* ESTRUTURAS DE DADOS */
typedef struct {
    int customer_id;
    int invoice_no;
    long date_days;     /* data em “dias desde 1970-01-01” */
    double total_price;
} SaleRow;

typedef struct {
    int id;
    long last_purchase; /* maior date_days observado */
    int *unique_invoices;
    int   n_invoices;
    int   cap_invoices;
    double monetary;
    int   frequency;    /* será tamanho de unique_invoices */
    long  recency;      /* (data_ref - last_purchase) */
} CustomerRFM;

typedef struct {
    double recency;
    double frequency;
    double monetary;
} RFMPoint;

/* FUNÇÕES AUXILIARES DE PARSE DE DATA */
static int is_leap(int y) {
    return ( (y%4 == 0 && y%100 != 0) || (y%400 == 0) );
}

/* Converte “YYYY-MM-DD hh:mm:ss” em número de dias desde 1970-01-01 (UTC). */
long date_to_days(const char *datestr) {
    int Y, M, D;
    /* só extraímos ano, mês e dia; horas/minutos/segundos ignorados para o cálculo de dias */
    if (sscanf(datestr, "%d-%d-%d", &Y, &M, &D) != 3) {
        return 0;
    }
    /* algoritmo simplificado: converte para julian days e subtrai base de 1970-01-01 */
    int a = (14 - M) / 12;
    int y = Y + 4800 - a;
    int m = M + 12*a - 3;
    long julian = D + (153*m + 2)/5 + 365L*y + y/4 - y/100 + y/400 - 32045;
    /* julian day de 1970-01-01 é 2440588 */
    return (long)(julian - 2440588);
}

/* Encontra ou insere um cliente em vetor sorted_client_ids; retorna índice */
int find_or_insert_customer(int *ids, int *n_cust, int cust_id) {
    for (int i = 0; i < *n_cust; i++) {
        if (ids[i] == cust_id) return i;
    }
    /* não achou, insere no fim */
    int idx = *n_cust;
    ids[idx] = cust_id;
    (*n_cust)++;
    return idx;
}

/* ADICIONA um número de Invoice único ao CustomerRFM (evita duplicatas via pesquisa linear). */
void add_invoice(CustomerRFM *cust, int invoice_no) {
    /* busca linear */
    for (int i = 0; i < cust->n_invoices; i++) {
        if (cust->unique_invoices[i] == invoice_no) return;
    }
    /* se não existe, adiciona */
    if (cust->n_invoices >= cust->cap_invoices) {
        cust->cap_invoices *= 2;
        cust->unique_invoices = realloc(cust->unique_invoices,
                                        cust->cap_invoices * sizeof(int));
    }
    cust->unique_invoices[cust->n_invoices++] = invoice_no;
}

/* Lê o CSV “cleaned_retail.csv” e preenche vetor de RFM por cliente.
 * Retorna número de clientes únicos e data_ref (maior data + 1 dia) via ponteiro. */
CustomerRFM *build_rfm(const char *csv_path, int *out_n_cust, long *out_data_ref) {
    FILE *f = fopen(csv_path, "r");
    if (!f) {
        fprintf(stderr, "Erro ao abrir '%s'\n", csv_path);
        exit(1);
    }

    /* Alocar buffers temporários */
    char line[MAX_LINE_LEN];
    char *token;
    int n_rows = 0;
    SaleRow *rows = malloc(MAX_ROWS * sizeof(SaleRow));
    if (!rows) {
        fprintf(stderr, "Memória insuficiente para linhas\n");
        exit(1);
    }

    /* Pular cabeçalho */
    if (!fgets(line, MAX_LINE_LEN, f)) {
        fprintf(stderr, "CSV vazio\n");
        exit(1);
    }

    /* Lê cada linha */
    while (fgets(line, MAX_LINE_LEN, f)) {
        if (n_rows >= MAX_ROWS) break;
        /* Exemplo de colunas (cleaned_retail.csv):
         * InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
         */
        /* Vamos fazer um parse simplificado com strtok em vírgula */
        int field = 0;
        SaleRow sr = {0};
        double unit_price = 0.0;
        int quantity = 0;
        char invoice_date_str[64] = "";
        int cust_id = 0;
        int invoice_no = 0;

        token = strtok(line, ",");
        while (token) {
            switch (field) {
                case 0:  /* InvoiceNo */
                    invoice_no = atoi(token);
                    break;
                case 3:  /* Quantity */
                    quantity = atoi(token);
                    break;
                case 4:  /* InvoiceDate */
                    strncpy(invoice_date_str, token, 63);
                    invoice_date_str[63] = '\0';
                    break;
                case 5:  /* UnitPrice */
                    unit_price = atof(token);
                    break;
                case 6:  /* CustomerID */
                    cust_id = atoi(token);
                    break;
                /* ignoramos outras colunas */
            }
            field++;
            token = strtok(NULL, ",");
        }

        /* Filtra conforme Python: CustomerID não nulo, Quantity>0, UnitPrice>0 */
        if (cust_id <= 0 || quantity <= 0 || unit_price <= 0.0) {
            continue;
        }

        sr.customer_id = cust_id;
        sr.invoice_no = invoice_no;
        sr.date_days = date_to_days(invoice_date_str);
        sr.total_price = ((double)quantity) * unit_price;
        rows[n_rows++] = sr;
    }
    fclose(f);

    /* Encontrar data_ref = max(date_days) + 1 */
    long max_date = 0;
    for (int i = 0; i < n_rows; i++) {
        if (rows[i].date_days > max_date) {
            max_date = rows[i].date_days;
        }
    }
    long data_ref = max_date + 1;

    /* Agrupar por CustomerID: primeiro extraímos todos os CustomerID únicos */
    int *cust_ids = malloc(MAX_CUSTOMERS * sizeof(int));
    if (!cust_ids) exit(1);
    int n_cust = 0;
    for (int i = 0; i < n_rows; i++) {
        find_or_insert_customer(cust_ids, &n_cust, rows[i].customer_id);
    }

    /* Alocamos vetor de CustomerRFM */
    CustomerRFM *rfm = malloc(n_cust * sizeof(CustomerRFM));
    if (!rfm) exit(1);
    for (int i = 0; i < n_cust; i++) {
        rfm[i].id = cust_ids[i];
        rfm[i].last_purchase = 0;
        rfm[i].cap_invoices = 8;
        rfm[i].unique_invoices = malloc(rfm[i].cap_invoices * sizeof(int));
        rfm[i].n_invoices = 0;
        rfm[i].monetary = 0.0;
    }

    /* Preenche cada CustomerRFM usando as linhas */
    for (int i = 0; i < n_rows; i++) {
        int idx = -1;
        /* encontra índice de CustomerID em cust_ids (poderia usar um map, mas fazemos busca linear) */
        for (int j = 0; j < n_cust; j++) {
            if (rfm[j].id == rows[i].customer_id) {
                idx = j;
                break;
            }
        }
        if (idx < 0) continue;  /* não deveria acontecer */

        /* Atualiza last_purchase se for maior */
        if (rows[i].date_days > rfm[idx].last_purchase) {
            rfm[idx].last_purchase = rows[i].date_days;
        }
        /* Adiciona invoice único */
        add_invoice(&rfm[idx], rows[i].invoice_no);
        /* Soma monetary */
        rfm[idx].monetary += rows[i].total_price;
    }
    free(rows);
    free(cust_ids);

    /* Agora, calcule frequency e recency para cada cliente */
    for (int i = 0; i < n_cust; i++) {
        rfm[i].frequency = rfm[i].n_invoices;
        rfm[i].recency = data_ref - rfm[i].last_purchase;
    }

    *out_n_cust = n_cust;
    *out_data_ref = data_ref;
    return rfm;
}

/* AMOSTRAGEM ALEATÓRIA (Fisher–Yates) para cortar a RFM para n_max clientes */
void sample_rfm(CustomerRFM **rfm_ptr, int *n_cust_ptr) {
    int n = *n_cust_ptr;
    if (n <= N_MAX) return;
    srand(SEED);
    /* Embaralha índices */
    for (int i = n - 1; i > 0; i--) {
        int j = rand() % (i + 1);
        /* swap em structures */
        CustomerRFM tmp = (*rfm_ptr)[i];
        (*rfm_ptr)[i] = (*rfm_ptr)[j];
        (*rfm_ptr)[j] = tmp;
    }
    /* corta para N_MAX */
    *n_cust_ptr = N_MAX;
    *rfm_ptr = realloc(*rfm_ptr, N_MAX * sizeof(CustomerRFM));
}

/* Converte vetor de CustomerRFM em matriz de pontos RFM (double) */
RFMPoint *build_rfm_points(CustomerRFM *rfm, int n_cust) {
    RFMPoint *pts = malloc(n_cust * sizeof(RFMPoint));
    for (int i = 0; i < n_cust; i++) {
        pts[i].recency   = (double) rfm[i].recency;
        pts[i].frequency = (double) rfm[i].frequency;
        pts[i].monetary  = rfm[i].monetary;
    }
    return pts;
}

/* StandardScaler: calcula média e desvio padrão de cada dimensão e padroniza in-place */
void standard_scaler(RFMPoint *X, int n, double *mean_out, double *std_out) {
    double sum_r = 0, sum_f = 0, sum_m = 0;
    for (int i = 0; i < n; i++) {
        sum_r += X[i].recency;
        sum_f += X[i].frequency;
        sum_m += X[i].monetary;
    }
    double mean_r = sum_r / n;
    double mean_f = sum_f / n;
    double mean_m = sum_m / n;
    /* calcula soma dos quadrados para variância */
    double sq_r = 0, sq_f = 0, sq_m = 0;
    for (int i = 0; i < n; i++) {
        sq_r += (X[i].recency   - mean_r) * (X[i].recency   - mean_r);
        sq_f += (X[i].frequency - mean_f) * (X[i].frequency - mean_f);
        sq_m += (X[i].monetary  - mean_m) * (X[i].monetary  - mean_m);
    }
    double std_r = sqrt(sq_r / n);
    double std_f = sqrt(sq_f / n);
    double std_m = sqrt(sq_m / n);

    /* Aplicar padronização */
    for (int i = 0; i < n; i++) {
        X[i].recency   = (X[i].recency   - mean_r) / (std_r + 1e-9);
        X[i].frequency = (X[i].frequency - mean_f) / (std_f + 1e-9);
        X[i].monetary  = (X[i].monetary  - mean_m) / (std_m + 1e-9);
    }
    mean_out[0] = mean_r; mean_out[1] = mean_f; mean_out[2] = mean_m;
    std_out[0]  = std_r;  std_out[1]  = std_f;  std_out[2]  = std_m;
}

/* Distância Euclidiana entre dois pontos RFM */
static inline double euclid_dist(const RFMPoint *a, const RFMPoint *b) {
    double dr = a->recency   - b->recency;
    double df = a->frequency - b->frequency;
    double dm = a->monetary  - b->monetary;
    return sqrt(dr*dr + df*df + dm*dm);
}

/* INICIALIZAÇÃO de centróides (KMeans++) seria ideal, mas aqui usamos inicialização aleatória simples */
void init_centroids(RFMPoint *X, int n, RFMPoint *centroids, int k) {
    srand(SEED);
    /* escolhe k índices distintos ao acaso */
    for (int i = 0; i < k; i++) {
        int idx = rand() % n;
        centroids[i] = X[idx];
    }
}

/* RODA K-Means com k clusters, retornando labels em vetor pré-alocado (tamanho n). */
void kmeans(const RFMPoint *X, int n, int k, int *labels) {
    RFMPoint *centroids = malloc(k * sizeof(RFMPoint));
    init_centroids((RFMPoint *)X, n, centroids, k);

    int *counts = malloc(k * sizeof(int));
    RFMPoint *sums  = malloc(k * sizeof(RFMPoint));
    int changed = 1;
    int it = 0;

    while (changed && it < MAX_KMEANS_IT) {
        it++;
        changed = 0;
        /* Passo 1: atribui cada ponto ao centroide mais próximo */
        for (int i = 0; i < n; i++) {
            double bestd = 1e300;
            int bestc = 0;
            for (int c = 0; c < k; c++) {
                double d = euclid_dist(&X[i], &centroids[c]);
                if (d < bestd) {
                    bestd = d;
                    bestc = c;
                }
            }
            if (labels[i] != bestc) {
                changed = 1;
                labels[i] = bestc;
            }
        }
        /* Passo 2: recalcula centróides */
        for (int c = 0; c < k; c++) {
            counts[c] = 0;
            sums[c].recency   = 0.0;
            sums[c].frequency = 0.0;
            sums[c].monetary  = 0.0;
        }
        for (int i = 0; i < n; i++) {
            int c = labels[i];
            sums[c].recency   += X[i].recency;
            sums[c].frequency += X[i].frequency;
            sums[c].monetary  += X[i].monetary;
            counts[c]++;
        }
        for (int c = 0; c < k; c++) {
            if (counts[c] > 0) {
                centroids[c].recency   = sums[c].recency   / counts[c];
                centroids[c].frequency = sums[c].frequency / counts[c];
                centroids[c].monetary  = sums[c].monetary  / counts[c];
            }
        }
    }
    free(centroids);
    free(counts);
    free(sums);
}

/* CÁLCULO SERIAL do Índice de Dunn:
 * - deltas: min distância entre qualquer par de pontos de clusters distintos
 * - diameters: max distância entre qualquer par de pontos dentro de cada cluster
 */
double dunn_index_serial(const RFMPoint *X, int *labels, int n, int k) {
    double min_inter = 1e300;
    double max_intra = 0.0;

    /* Para cada par (c1, c2), percorre pontos i∈c1, j∈c2 → atualiza min_inter */
    for (int c1 = 0; c1 < k; c1++) {
        for (int c2 = c1 + 1; c2 < k; c2++) {
            /* varre todos os pares i,j */
            for (int i = 0; i < n; i++) {
                if (labels[i] != c1) continue;
                for (int j = 0; j < n; j++) {
                    if (labels[j] != c2) continue;
                    double d = euclid_dist(&X[i], &X[j]);
                    if (d < min_inter) {
                        min_inter = d;
                    }
                }
            }
        }
    }

    /* Para cada cluster c, percorre todos os pares internos i,j para achar diâmetro */
    for (int c = 0; c < k; c++) {
        for (int i = 0; i < n; i++) {
            if (labels[i] != c) continue;
            for (int j = i + 1; j < n; j++) {
                if (labels[j] != c) continue;
                double d = euclid_dist(&X[i], &X[j]);
                if (d > max_intra) {
                    max_intra = d;
                }
            }
        }
    }

    if (max_intra < 1e-12) return 0.0;
    return min_inter / max_intra;
}

/* CÁLCULO PARALELO (com OpenMP) do índice de Dunn.
 * Paralelizamos dupla de clusters e laços internos de pontos.
 */
double dunn_index_parallel(const RFMPoint *X, int *labels, int n, int k) {
    double min_inter = 1e300;
    double max_intra = 0.0;

    /* 1) Calcular min_inter paralelo:
     * cada iteração de (c1,c2) pode ser feita em paralelo
     */
    #pragma omp parallel
    {
        double local_min_inter = 1e300;
        #pragma omp for schedule(dynamic)
        for (int c1 = 0; c1 < k; c1++) {
            for (int c2 = c1 + 1; c2 < k; c2++) {
                for (int i = 0; i < n; i++) {
                    if (labels[i] != c1) continue;
                    for (int j = 0; j < n; j++) {
                        if (labels[j] != c2) continue;
                        double d = euclid_dist(&X[i], &X[j]);
                        if (d < local_min_inter) {
                            local_min_inter = d;
                        }
                    }
                }
            }
        }
        #pragma omp critical
        {
            if (local_min_inter < min_inter) {
                min_inter = local_min_inter;
            }
        }
    }

    /* 2) Calcular max_intra paralelo:
     * cada cluster c pode ser avaliado em paralelo
     */
    #pragma omp parallel
    {
        double local_max_intra = 0.0;
        #pragma omp for schedule(dynamic)
        for (int c = 0; c < k; c++) {
            for (int i = 0; i < n; i++) {
                if (labels[i] != c) continue;
                for (int j = i + 1; j < n; j++) {
                    if (labels[j] != c) continue;
                    double d = euclid_dist(&X[i], &X[j]);
                    if (d > local_max_intra) {
                        local_max_intra = d;
                    }
                }
            }
        }
        #pragma omp critical
        {
            if (local_max_intra > max_intra) {
                max_intra = local_max_intra;
            }
        }
    }

    if (max_intra < 1e-12) return 0.0;
    return min_inter / max_intra;
}

int main() {
    double t0, t1;

    printf("📥 Carregando 'cleaned_retail.csv' e calculando RFM...\n");
    int n_cust;
    long data_ref;
    CustomerRFM *rfm = build_rfm("cleaned_retail.csv", &n_cust, &data_ref);
    printf("  → Clientes únicos antes da amostragem: %d\n", n_cust);

    /* Se excede N_MAX, faz amostragem aleatória */
    if (n_cust > N_MAX) {
        sample_rfm(&rfm, &n_cust);
        printf("  → Após amostragem aleatória para N_MAX=%d, clientes: %d\n", N_MAX, n_cust);
    }

    /* Monta pontos RFM */
    RFMPoint *X = build_rfm_points(rfm, n_cust);

    /* Padroniza */
    double mean_arr[3], std_arr[3];
    standard_scaler(X, n_cust, mean_arr, std_arr);
    printf("✅ RFM processado e padronizado (%d clientes).\n\n", n_cust);

    /* Prepara vetor de labels */
    int *labels = malloc(n_cust * sizeof(int));

    /* Para cada k = 4..11, executa KMeans e Dunn (serial & paralelo) */
    printf("🧪 Iniciando experimentos (k = 4..11)\n\n");
    for (int k = 4; k <= 11; k++) {
        printf("============================================================\n");
        printf("🔬 Experimento com k = %d\n", k);
        printf("============================================================\n");

        /* Inicializa labels com -1 */
        for (int i = 0; i < n_cust; i++) labels[i] = -1;

        /* Roda KMeans */
        t0 = omp_get_wtime();
        kmeans(X, n_cust, k, labels);
        t1 = omp_get_wtime();
        double t_kmeans = t1 - t0;
        printf("   ✅ KMeans (k=%d) concluído em %.2fs\n", k, t_kmeans);

        /* Cálculo serial do Dunn index */
        printf("   ⏱ [Serial] Calculando Dunn index...\n");
        t0 = omp_get_wtime();
        double d_s = dunn_index_serial(X, labels, n_cust, k);
        t1 = omp_get_wtime();
        double t_s = t1 - t0;
        printf("     → Dunn serial = %.4f | Tempo: %.2fs\n", d_s, t_s);

        /* Cálculo paralelo do Dunn index */
        printf("   ⏱ [Parallel] Calculando Dunn index...\n");
        t0 = omp_get_wtime();
        double d_p = dunn_index_parallel(X, labels, n_cust, k);
        t1 = omp_get_wtime();
        double t_p = t1 - t0;
        printf("     → Dunn paralelo = %.4f | Tempo: %.2fs\n", d_p, t_p);

        double speedup = (t_p > 1e-9) ? (t_s / t_p) : 0.0;
        printf("   ⚡ Speedup = %.2fx\n\n", speedup);
    }

    /* Libera memória */
    for (int i = 0; i < n_cust; i++) {
        free(rfm[i].unique_invoices);
    }
    free(rfm);
    free(X);
    free(labels);

    printf("📊 Todos os experimentos concluídos.\n");
    return 0;
}

Writing dunn_openmp.c


In [ ]:
% gcc -O2 -fopenmp -o dunn_openmp dunn_openmp.c -lm
% ./dunn_openmp

📥 Carregando 'cleaned_retail.csv' e calculando RFM...
  → Clientes únicos antes da amostragem: 4129
✅ RFM processado e padronizado (4129 clientes).

🧪 Iniciando experimentos (k = 4..11)

🔬 Experimento com k = 4
   ✅ KMeans (k=4) concluído em 0.00s
   ⏱ [Serial] Calculando Dunn index...
     → Dunn serial = 0.0004 | Tempo: 0.02s
   ⏱ [Parallel] Calculando Dunn index...
     → Dunn paralelo = 0.0004 | Tempo: 0.03s
   ⚡ Speedup = 0.76x

🔬 Experimento com k = 5
   ✅ KMeans (k=5) concluído em 0.01s
   ⏱ [Serial] Calculando Dunn index...
     → Dunn serial = 0.0002 | Tempo: 0.04s
   ⏱ [Parallel] Calculando Dunn index...
     → Dunn paralelo = 0.0002 | Tempo: 0.05s
   ⚡ Speedup = 0.88x

🔬 Experimento com k = 6
   ✅ KMeans (k=6) concluído em 0.00s
   ⏱ [Serial] Calculando Dunn index...
     → Dunn serial = 0.0001 | Tempo: 0.06s
   ⏱ [Parallel] Calculando Dunn index...
     → Dunn paralelo = 0.0001 | Tempo: 0.07s
   ⚡ Speedup = 0.82x

🔬 Experimento com k = 7
   ✅ KMeans (k=7) concluído em 0.01s

## CUDA

### Arrumando versão CUDA

In [ ]:
%nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%apt-get update -y
%apt-get install -y cuda-12-4

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,740 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,741 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [ ]:
%sudo rm -rf /usr/local/cuda
%sudo ln -s /usr/local/cuda-12.4 /usr/local/cuda

In [ ]:
%export PATH=/usr/local/cuda-12.4/bin:$PATH
%export LD_LIBRARY_PATH=/usr/local/cuda-12.4/lib64:$LD_LIBRARY_PATH

In [ ]:
%pip install nvcc4jupyter

In [8]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmphvoyrb2u".


### Código

In [26]:
%%writefile dunn_cuda_fixed.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>
#include <time.h>

// CUDA Runtime
#include <cuda_runtime.h>

/* CONFIGURAÇÕES GLOBAIS */
#define MAX_LINE_LEN   4096
#define MAX_CUSTOMERS  5000    /* tamanho máximo esperado de clientes únicos */
#define MAX_ROWS       300000  /* número máximo de linhas no CSV */
#define N_MAX          300000  /* limite de RFM após amostragem */
#define MAX_KMEANS_IT  100     /* número máximo de iterações no KMeans */
#define SEED           42

/* ESTRUTURAS DE DADOS */
typedef struct {
    int customer_id;
    int invoice_no;
    long date_days;     /* data em “dias desde 1970-01-01” */
    double total_price;
} SaleRow;

typedef struct {
    int id;
    long last_purchase; /* maior date_days observado */
    int *unique_invoices;
    int   n_invoices;
    int   cap_invoices;
    double monetary;
    int   frequency;    /* será tamanho de unique_invoices */
    long  recency;      /* (data_ref - last_purchase) */
} CustomerRFM;

typedef struct {
    double recency;
    double frequency;
    double monetary;
} RFMPoint;

/*-------------------------------------*/
/* FUNÇÕES AUXILIARES (CPU)         */
/*-------------------------------------*/

/* Converte “YYYY-MM-DD hh:mm:ss” em número de dias desde 1970-01-01 (UTC). */
// Removed unused is_leap function
long date_to_days(const char *datestr) {
    int Y, M, D;
    if (sscanf(datestr, "%d-%d-%d", &Y, &M, &D) != 3) {
        return 0;
    }
    int a = (14 - M) / 12;
    int y = Y + 4800 - a;
    int m = M + 12*a - 3;
    long julian = D + (153*m + 2)/5 + 365L*y + y/4 - y/100 + y/400 - 32045;
    return (long)(julian - 2440588);
}

/* Encontra ou insere um cliente em vetor sorted_client_ids; retorna índice */
int find_or_insert_customer(int *ids, int *n_cust, int cust_id) {
    for (int i = 0; i < *n_cust; i++) {
        if (ids[i] == cust_id) return i;
    }
    int idx = *n_cust;
    ids[idx] = cust_id;
    (*n_cust)++;
    return idx;
}

/* ADICIONA um número de Invoice único ao CustomerRFM */
void add_invoice(CustomerRFM *cust, int invoice_no) {
    for (int i = 0; i < cust->n_invoices; i++) {
        if (cust->unique_invoices[i] == invoice_no) return;
    }
    if (cust->n_invoices >= cust->cap_invoices) {
        cust->cap_invoices *= 2;
        cust->unique_invoices = (int *)realloc(cust->unique_invoices,
                                        cust->cap_invoices * sizeof(int));
    }
    cust->unique_invoices[cust->n_invoices++] = invoice_no;
}

/* Lê o CSV “cleaned_retail.csv” e preenche vetor de RFM por cliente */
CustomerRFM *build_rfm(const char *csv_path, int *out_n_cust, long *out_data_ref) {
    FILE *f = fopen(csv_path, "r");
    if (!f) {
        fprintf(stderr, "Erro ao abrir '%s'\n", csv_path);
        exit(1);
    }
    char line[MAX_LINE_LEN];
    char *token;
    int n_rows = 0;
    SaleRow *rows = (SaleRow *)malloc(MAX_ROWS * sizeof(SaleRow));
    if (!rows) { fprintf(stderr, "Memória insuficiente para linhas\n"); exit(1); }

    /* Pular cabeçalho */
    if (!fgets(line, MAX_LINE_LEN, f)) {
        fprintf(stderr, "CSV vazio\n");
        exit(1);
    }

    while (fgets(line, MAX_LINE_LEN, f)) {
        if (n_rows >= MAX_ROWS) break;
        int field = 0;
        SaleRow sr = {0};
        double unit_price = 0.0;
        int quantity = 0;
        char invoice_date_str[64] = "";
        int cust_id = 0;
        int invoice_no = 0;

        token = strtok(line, ",");
        while (token) {
            switch (field) {
                case 0:  invoice_no = atoi(token); break;
                case 3:  quantity = atoi(token); break;
                case 4:
                    strncpy(invoice_date_str, token, 63);
                    invoice_date_str[63] = '\0';
                    break;
                case 5:  unit_price = atof(token); break;
                case 6:  cust_id = atoi(token); break;
            }
            field++;
            token = strtok(NULL, ",");
        }
        if (cust_id <= 0 || quantity <= 0 || unit_price <= 0.0) {
            continue;
        }
        sr.customer_id = cust_id;
        sr.invoice_no = invoice_no;
        sr.date_days = date_to_days(invoice_date_str);
        sr.total_price = ((double)quantity) * unit_price;
        rows[n_rows++] = sr;
    }
    fclose(f);

    long max_date = 0;
    for (int i = 0; i < n_rows; i++) {
        if (rows[i].date_days > max_date) max_date = rows[i].date_days;
    }
    long data_ref = max_date + 1;

    int *cust_ids = (int *)malloc(MAX_CUSTOMERS * sizeof(int));
    if (!cust_ids) exit(1);
    int n_cust = 0;
    for (int i = 0; i < n_rows; i++) {
        find_or_insert_customer(cust_ids, &n_cust, rows[i].customer_id);
    }

    CustomerRFM *rfm = (CustomerRFM *)malloc(n_cust * sizeof(CustomerRFM));
    if (!rfm) exit(1);
    for (int i = 0; i < n_cust; i++) {
        rfm[i].id = cust_ids[i];
        rfm[i].last_purchase = 0;
        rfm[i].cap_invoices = 8;
        rfm[i].unique_invoices = (int *)malloc(rfm[i].cap_invoices * sizeof(int));
        rfm[i].n_invoices = 0;
        rfm[i].monetary = 0.0;
    }

    for (int i = 0; i < n_rows; i++) {
        int idx = -1;
        for (int j = 0; j < n_cust; j++) {
            if (rfm[j].id == rows[i].customer_id) {
                idx = j; break;
            }
        }
        if (idx < 0) continue;
        if (rows[i].date_days > rfm[idx].last_purchase) {
            rfm[idx].last_purchase = rows[i].date_days;
        }
        add_invoice(&rfm[idx], rows[i].invoice_no);
        rfm[idx].monetary += rows[i].total_price;
    }
    free(rows);
    free(cust_ids);

    for (int i = 0; i < n_cust; i++) {
        rfm[i].frequency = rfm[i].n_invoices;
        rfm[i].recency = data_ref - rfm[i].last_purchase;
    }

    *out_n_cust = n_cust;
    *out_data_ref = data_ref;
    return rfm;
}

/* AMOSTRAGEM ALEATÓRIA (Fisher–Yates) para cortar a RFM */
void sample_rfm(CustomerRFM **rfm_ptr, int *n_cust_ptr) {
    int n = *n_cust_ptr;
    if (n <= N_MAX) return;
    srand(SEED);
    for (int i = n - 1; i > 0; i--) {
        int j = rand() % (i + 1);
        CustomerRFM tmp = (*rfm_ptr)[i];
        (*rfm_ptr)[i] = (*rfm_ptr)[j];
        (*rfm_ptr)[j] = tmp;
    }
    *n_cust_ptr = N_MAX;
    *rfm_ptr = (CustomerRFM *)realloc(*rfm_ptr, N_MAX * sizeof(CustomerRFM));
}

/* Converte vetor de CustomerRFM em matriz de pontos RFM (double) */
RFMPoint *build_rfm_points(CustomerRFM *rfm, int n_cust) {
    RFMPoint *pts = (RFMPoint *)malloc(n_cust * sizeof(RFMPoint));
    for (int i = 0; i < n_cust; i++) {
        pts[i].recency   = (double) rfm[i].recency;
        pts[i].frequency = (double) rfm[i].frequency;
        pts[i].monetary  = rfm[i].monetary;
    }
    return pts;
}

/* StandardScaler: calcula média e desvio padrão; padroniza in-place */
void standard_scaler(RFMPoint *X, int n, double *mean_out, double *std_out) {
    double sum_r = 0, sum_f = 0, sum_m = 0;
    for (int i = 0; i < n; i++) {
        sum_r += X[i].recency;
        sum_f += X[i].frequency;
        sum_m += X[i].monetary;
    }
    double mean_r = sum_r / n;
    double mean_f = sum_f / n;
    double mean_m = sum_m / n;

    double sq_r = 0, sq_f = 0, sq_m = 0;
    for (int i = 0; i < n; i++) {
        sq_r += (X[i].recency   - mean_r) * (X[i].recency   - mean_r);
        sq_f += (X[i].frequency - mean_f) * (X[i].frequency - mean_f);
        sq_m += (X[i].monetary  - mean_m) * (X[i].monetary  - mean_m);
    }
    double std_r = sqrt(sq_r / n);
    double std_f = sqrt(sq_f / n);
    double std_m = sqrt(sq_m / n);

    for (int i = 0; i < n; i++) {
        X[i].recency   = (X[i].recency   - mean_r) / (std_r + 1e-9);
        X[i].frequency = (X[i].frequency - mean_f) / (std_f + 1e-9);
        X[i].monetary  = (X[i].monetary  - mean_m) / (std_m + 1e-9);
    }
    mean_out[0] = mean_r; mean_out[1] = mean_f; mean_out[2] = mean_m;
    std_out[0]  = std_r;  std_out[1]  = std_f;  std_out[2]  = std_m;
}

/* Distância Euclidiana entre dois pontos RFM (versão CPU) */
static inline double euclid_dist(const RFMPoint *a, const RFMPoint *b) {
    double dr = a->recency   - b->recency;
    double df = a->frequency - b->frequency;
    double dm = a->monetary  - b->monetary;
    return sqrt(dr*dr + df*df + dm*dm);
}

/* INICIALIZAÇÃO e execução de KMeans (CPU, serial) */
void init_centroids(RFMPoint *X, int n, RFMPoint *centroids, int k) {
    srand(SEED);
    for (int i = 0; i < k; i++) {
        int idx = rand() % n;
        centroids[i] = X[idx];
    }
}

void kmeans(const RFMPoint *X, int n, int k, int *labels) {
    RFMPoint *centroids = (RFMPoint *)malloc(k * sizeof(RFMPoint));
    init_centroids((RFMPoint *)X, n, centroids, k);

    int *counts = (int *)malloc(k * sizeof(int));
    RFMPoint *sums  = (RFMPoint *)malloc(k * sizeof(RFMPoint));
    int changed = 1;
    int it = 0;

    while (changed && it < MAX_KMEANS_IT) {
        it++;
        changed = 0;
        for (int i = 0; i < n; i++) {
            double bestd = 1e300;
            int bestc = 0;
            for (int c = 0; c < k; c++) {
                double d = euclid_dist(&X[i], &centroids[c]);
                if (d < bestd) {
                    bestd = d;
                    bestc = c;
                }
            }
            if (labels[i] != bestc) {
                changed = 1;
                labels[i] = bestc;
            }
        }
        for (int c = 0; c < k; c++) {
            counts[c] = 0;
            sums[c].recency   = 0.0;
            sums[c].frequency = 0.0;
            sums[c].monetary  = 0.0;
        }
        for (int i = 0; i < n; i++) {
            int c = labels[i];
            sums[c].recency   += X[i].recency;
            sums[c].frequency += X[i].frequency;
            sums[c].monetary  += X[i].monetary;
            counts[c]++;
        }
        for (int c = 0; c < k; c++) {
            if (counts[c] > 0) {
                centroids[c].recency   = sums[c].recency   / counts[c];
                centroids[c].frequency = sums[c].frequency / counts[c];
                centroids[c].monetary  = sums[c].monetary  / counts[c];
            }
        }
    }
    free(centroids);
    free(counts);
    free(sums);
}

/*-------------------------------------*/
/* FUNÇÕES CUDA (DEVICE + KERNEL)  */
/*-------------------------------------*/

/* Estrutura equivalente a RFMPoint na GPU */
typedef struct {
    double recency;
    double frequency;
    double monetary;
} RFMPointGPU;

/* Função inline no device para calcular distância Euclidiana */
__device__ double dev_euclid_dist(const RFMPointGPU *a, const RFMPointGPU *b) {
    double dr = a->recency   - b->recency;
    double df = a->frequency - b->frequency;
    double dm = a->monetary  - b->monetary;
    return sqrt(dr*dr + df*df + dm*dm);
}

/*
  Kernel para computar, para cada par de clusters (c1, c2),
  a menor distância entre qualquer ponto de c1 e qualquer ponto de c2.

  - d_X: vetor de RFMPointGPU (tamanho n), com todos os pontos
  - d_labels: vetor de rótulos (tamanho n)
  - n: número total de pontos
  - d_pair_c1, d_pair_c2: vetores de mesmo tamanho numPairs, indicando
    para cada índice de bloco qual par (c1, c2) ele processa
  - numPairs: número total de pares (k*(k-1)/2)
  - d_out: vetor de saída (tamanho numPairs) onde cada posição receberá
    a menor distância encontrada para aquele par de clusters
*/
extern __shared__ double sdata_min[]; // Shared memory para redução do min

__global__
void kernel_min_inter(const RFMPointGPU *d_X,
                      const int *d_labels,
                      int n,
                      const int *d_pair_c1,
                      const int *d_pair_c2,
                      int numPairs,
                      double *d_out)
{
    int pairIdx = blockIdx.x;
    if (pairIdx >= numPairs) return;

    int c1 = d_pair_c1[pairIdx];
    int c2 = d_pair_c2[pairIdx];

    double local_min = 1e300;
    int tid = threadIdx.x;
    int blockSize = blockDim.x;

    for (int i = tid; i < n; i += blockSize) {
        if (d_labels[i] != c1) continue;
        for (int j = 0; j < n; j++) {
            if (d_labels[j] != c2) continue;
            double d = dev_euclid_dist(&d_X[i], &d_X[j]);
            if (d == 0.0) continue;  // <- ignorar pontos idênticos!
            if (d < local_min) local_min = d;
        }
    }

    extern __shared__ double sdata_min[];
    sdata_min[tid] = local_min;
    __syncthreads();

    for (int stride = blockSize / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            double other = sdata_min[tid + stride];
            if (other < sdata_min[tid]) sdata_min[tid] = other;
        }
        __syncthreads();
    }

    if (tid == 0) {
        d_out[pairIdx] = (sdata_min[0] == 1e300 ? NAN : sdata_min[0]);
    }
}

/*
  Kernel para computar, para cada cluster c,
  o maior diâmetro (distância máxima entre quaisquer dois pontos dentro de c).

  - d_X: vetor de RFMPointGPU (tamanho n)
  - d_labels: vetor de rótulos (tamanho n)
  - n: número total de pontos
  - k: número de clusters
  - d_out: vetor de saída (tamanho k) onde cada posição receberá
    o maior diâmetro interno daquele cluster c
*/
extern __shared__ double sdata_max[]; // Shared memory para redução do max

__global__
void kernel_max_intra(const RFMPointGPU *d_X,
                      const int *d_labels,
                      int n,
                      int k,
                      double *d_out)
{
    int c = blockIdx.x; // cada bloco processa um cluster c
    if (c >= k) return;

    double local_max = 0.0;
    int tid = threadIdx.x;
    int blockSize = blockDim.x;

    // Cada thread varre uma parcela dos pontos i
    for (int i = tid; i < n; i += blockSize) {
        if (d_labels[i] != c) continue;
        // Para cada ponto j > i no mesmo cluster
        for (int j = i + 1; j < n; j++) {
            if (d_labels[j] != c) continue;
            double d = dev_euclid_dist(&d_X[i], &d_X[j]);
            if (d > local_max) {
                local_max = d;
            }
        }
    }
    // Armazena resultado parcial no shared memory
    sdata_max[tid] = local_max;
    __syncthreads();

    // Redução em bloco para achar o máximo
    for (int stride = blockSize / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            double o = sdata_max[tid + stride];
            if (o > sdata_max[tid]) {
                sdata_max[tid] = o;
            }
        }
        __syncthreads();
    }

    if (tid == 0) {
        d_out[c] = sdata_max[0];
    }
}

/*
  Função host para calcular o Dunn Index usando CUDA.
  Recebe X (RFMPoint em CPU), labels (em CPU), n (número de pontos) e k (número de clusters).
  Retorna o Dunn Index como double.
*/
double dunn_index_cuda(const RFMPoint *X_cpu, int *labels_cpu, int n, int k) {
    // 1) Preparar dados na GPU
    RFMPointGPU *d_X = NULL;
    int *d_labels = NULL;

    size_t size_X = n * sizeof(RFMPointGPU);
    size_t size_labels = n * sizeof(int);

    // Alocar memória na GPU
    cudaMalloc((void**)&d_X, size_X);
    cudaMalloc((void**)&d_labels, size_labels);

    // Copiar X para GPU, convertendo para RFMPointGPU
    RFMPointGPU *h_Xgpu = (RFMPointGPU*)malloc(size_X);
    for (int i = 0; i < n; i++) {
        h_Xgpu[i].recency   = X_cpu[i].recency;
        h_Xgpu[i].frequency = X_cpu[i].frequency;
        h_Xgpu[i].monetary  = X_cpu[i].monetary;
    }
    cudaMemcpy(d_X, h_Xgpu, size_X, cudaMemcpyHostToDevice);
    free(h_Xgpu);

    // Copiar labels para GPU
    cudaMemcpy(d_labels, labels_cpu, size_labels, cudaMemcpyHostToDevice);

    // 2) Preparar pares de clusters (c1, c2)
    int numPairs = k * (k - 1) / 2;
    if (numPairs <= 0 && k > 0) { // Handle k=1 case where numPairs would be 0
        // If k=1, Dunn index is typically undefined or 0.
        // Or, if you define it as 0 or some other value for k=1, handle here.
        // For now, let's avoid division by zero issues later if max_intra is also 0.
        // And avoid launching kernels with numPairs = 0.
        cudaFree(d_X);
        cudaFree(d_labels);
        if (k==1 && n > 0) { // If there's one cluster with points, max_intra might be > 0
             // Calculate max_intra for the single cluster
            double *d_max_intra_single = NULL;
            cudaMalloc((void**)&d_max_intra_single, k * sizeof(double)); // k is 1
            int threadsPerBlock_single = 256; // Or a suitable number
            if (n < threadsPerBlock_single) threadsPerBlock_single = n;
            if (threadsPerBlock_single == 0 && n > 0) threadsPerBlock_single = 1; // ensure at least 1 thread if n>0
            else if (threadsPerBlock_single == 0 && n == 0) threadsPerBlock_single = 1; // avoid 0 threads

            size_t sharedMemSize_max_single = threadsPerBlock_single * sizeof(double);
             if (threadsPerBlock_single > 0) {
                kernel_max_intra<<<1, threadsPerBlock_single, sharedMemSize_max_single>>>(
                    d_X, d_labels, n, k, d_max_intra_single
                );
                cudaDeviceSynchronize();
                double h_max_intra_single_val;
                cudaMemcpy(&h_max_intra_single_val, d_max_intra_single, sizeof(double), cudaMemcpyDeviceToHost);
                cudaFree(d_max_intra_single);
                // Dunn index for k=1 is often considered 0 or undefined.
                // If min_inter is conceptually infinite (no other cluster), and max_intra > 0,
                // this definition doesn't fit well. Often, Dunn isn't used for k=1.
                // Let's return 0.0 as a safe default.
                return 0.0;
            } else { // n == 0
                return 0.0; // No points, no clusters, Dunn index 0
            }
        }
        return 0.0; // For numPairs <= 0 (e.g. k=0 or k=1)
    }


    int *h_pair_c1 = (int*)malloc(numPairs * sizeof(int));
    int *h_pair_c2 = (int*)malloc(numPairs * sizeof(int));
    int idx = 0;
    for (int c1 = 0; c1 < k; c1++) {
        for (int c2 = c1 + 1; c2 < k; c2++) {
            if (idx < numPairs) { // Bounds check
                h_pair_c1[idx] = c1;
                h_pair_c2[idx] = c2;
                idx++;
            }
        }
    }

    int *d_pair_c1 = NULL, *d_pair_c2 = NULL;
    cudaMalloc((void**)&d_pair_c1, numPairs * sizeof(int));
    cudaMalloc((void**)&d_pair_c2, numPairs * sizeof(int));
    cudaMemcpy(d_pair_c1, h_pair_c1, numPairs * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_pair_c2, h_pair_c2, numPairs * sizeof(int), cudaMemcpyHostToDevice);

    double *d_min_inter = NULL;
    cudaMalloc((void**)&d_min_inter, numPairs * sizeof(double));

    // 3) Lançar kernel para min_inter
    int threadsPerBlock = 256;
    int blocksPerGrid_inter = numPairs;
    size_t sharedMemSize_min = threadsPerBlock * sizeof(double);
    if (numPairs > 0) { // Only launch if there are pairs to process
        kernel_min_inter<<<blocksPerGrid_inter, threadsPerBlock, sharedMemSize_min>>>(
            d_X, d_labels, n, d_pair_c1, d_pair_c2, numPairs, d_min_inter
        );
        cudaDeviceSynchronize();
    }


    // Copiar resultados de min_inter para host
    double *h_min_inter = (double*)malloc(numPairs * sizeof(double));
    if (numPairs > 0) {
        cudaMemcpy(h_min_inter, d_min_inter, numPairs * sizeof(double), cudaMemcpyDeviceToHost);
    }
    // Corrige: ignora NaN (pares sem pontos diferentes)
    double min_inter = 1e300;
    for (int i = 0; i < numPairs; i++) {
        if (!isnan(h_min_inter[i]) && h_min_inter[i] < min_inter) {
            min_inter = h_min_inter[i];
        }
    }
    // 4) Lançar kernel para max_intra
    double *d_max_intra = NULL;
    cudaMalloc((void**)&d_max_intra, k * sizeof(double));

    int blocksPerGrid_intra = k;
    size_t sharedMemSize_max = threadsPerBlock * sizeof(double);
    // Adjust threadsPerBlock if n is very small for kernel_max_intra
    if (n > 0 && n < threadsPerBlock) { // If total points are less than default threads
        // This adjustment is more for semantic correctness or minor optimization for small N.
        // The kernel itself has loops `i = tid; i < n; i += blockSize`
        // which correctly handle `n < blockSize`.
        // However, if shared memory allocation depends on a smaller effective block size:
        // threads_max_intra = n; // or next power of 2, e.g. 32, 64, 128...
        // For simplicity, keeping `threadsPerBlock` but ensuring `sharedMemSize_max` is sufficient.
        // The current shared memory allocation is based on `threadsPerBlock`, which is fine.
    } else if (n == 0) { // No points
        cudaFree(d_X);
        cudaFree(d_labels);
        if(d_pair_c1) cudaFree(d_pair_c1);
        if(d_pair_c2) cudaFree(d_pair_c2);
        if(d_min_inter) cudaFree(d_min_inter);
        cudaFree(d_max_intra);
        free(h_pair_c1);
        free(h_pair_c2);
        free(h_min_inter);
        return 0.0; // No points, Dunn index is 0
    }


    kernel_max_intra<<<blocksPerGrid_intra, threadsPerBlock, sharedMemSize_max>>>(
        d_X, d_labels, n, k, d_max_intra
    );
    cudaDeviceSynchronize();

    // Copiar resultados de max_intra para host
    double *h_max_intra = (double*)malloc(k * sizeof(double));
    cudaMemcpy(h_max_intra, d_max_intra, k * sizeof(double), cudaMemcpyDeviceToHost);

    // Encontrar o máximo global entre todos os clusters
    double max_intra = 0.0;
    for (int i = 0; i < k; i++) {
        if (h_max_intra[i] > max_intra) {
            max_intra = h_max_intra[i];
        }
    }

    // Limpeza de memória GPU e host temporária
    cudaFree(d_X);
    cudaFree(d_labels);
    if(d_pair_c1) cudaFree(d_pair_c1); // Free only if allocated
    if(d_pair_c2) cudaFree(d_pair_c2);
    if(d_min_inter) cudaFree(d_min_inter);
    cudaFree(d_max_intra);
    free(h_pair_c1);
    free(h_pair_c2);
    free(h_min_inter);
    free(h_max_intra);

    // 5) Calcular Dunn = min_inter / max_intra
    if (max_intra < 1e-12) { // Avoid division by zero or near-zero
        // If max_intra is zero (e.g., all points in a cluster are identical, or clusters have < 2 points),
        // the Dunn index could be considered infinite if min_inter > 0, or undefined.
        // Returning 0.0 is a common way to handle this degenerate case.
        // If min_inter is also 0, 0/0 is undefined, 0.0 is a safe return.
        return 0.0;
    }
    if (numPairs == 0 && k == 1) { // Special handling for k=1 already done
        return 0.0; // Consistent with earlier return for k=1.
    }

    double dunn = min_inter / max_intra;
    return dunn;
}

double dunn_index_serial(const RFMPoint *X, const int *labels, int n, int k) {
    double min_inter = 1e300, max_intra = 0.0;
    for (int c1 = 0; c1 < k; c1++) {
        // Diâmetro intra-cluster
        for (int i = 0; i < n; i++) {
            if (labels[i] != c1) continue;
            for (int j = i + 1; j < n; j++) {
                if (labels[j] != c1) continue;
                double d = euclid_dist(&X[i], &X[j]);
                if (d > max_intra) max_intra = d;
            }
        }

        // Distância mínima inter-cluster
        for (int c2 = c1 + 1; c2 < k; c2++) {
            for (int i = 0; i < n; i++) {
                if (labels[i] != c1) continue;
                for (int j = 0; j < n; j++) {
                    if (labels[j] != c2) continue;
                    double d = euclid_dist(&X[i], &X[j]);
                    if (d == 0.0) continue;  // ignora pontos idênticos
                    if (d < min_inter) min_inter = d;
                }
            }
        }
    }
    return (max_intra < 1e-12 ? 0.0 : min_inter / max_intra);
}

/*-------------------------------------*/
/* main                  */
/*-------------------------------------*/
int main() {
    double t0, t1;

    printf("📥 Carregando 'cleaned_retail.csv' e calculando RFM...\n");
    int n_cust;
    long data_ref;
    CustomerRFM *rfm = build_rfm("cleaned_retail.csv", &n_cust, &data_ref);
    if (n_cust == 0) {
        printf("  🔴 Nenhum cliente encontrado após o processamento do CSV. Encerrando.\n");
        if (rfm) free(rfm); // rfm might be allocated but empty or point to an empty array
        return 1;
    }
    printf("  → Clientes únicos antes da amostragem: %d\n", n_cust);


    if (n_cust > N_MAX) {
        sample_rfm(&rfm, &n_cust);
        printf("  → Após amostragem aleatória para N_MAX=%d, clientes: %d\n", N_MAX, n_cust);
    }
     if (n_cust == 0) { // Check again after sampling, though N_MAX should be > 0
        printf("  🔴 Nenhum cliente restante após a amostragem. Encerrando.\n");
        // rfm would have been realloc'd to N_MAX size, or still needs freeing if original was > N_MAX then sampled to 0 (unlikely with N_MAX > 0)
        // The unique_invoices within rfm elements also need freeing if rfm was populated before sampling to 0.
        // However, sample_rfm reallocs, so if n_cust becomes 0, *rfm_ptr points to a 0-sized block if N_MAX=0 (not the case here)
        // or a N_MAX-sized block if sampled down.
        // For simplicity, if n_cust is 0, the loops for freeing unique_invoices later won't run.
        free(rfm);
        return 1;
    }


    RFMPoint *X = build_rfm_points(rfm, n_cust);
    double mean_arr[3], std_arr[3];
    standard_scaler(X, n_cust, mean_arr, std_arr);
    printf("✅ RFM processado e padronizado (%d clientes).\n\n", n_cust);

    int *labels = (int*)malloc(n_cust * sizeof(int));
    if (!labels && n_cust > 0) {
        fprintf(stderr, "Falha ao alocar memória para labels.\n");
        // Free previously allocated memory
        for (int i = 0; i < n_cust; i++) { // n_cust might have changed after sampling
            if (rfm[i].unique_invoices) free(rfm[i].unique_invoices);
        }
        free(rfm);
        free(X);
        return 1;
    }


    printf("🧪 Iniciando experimentos (k = 4..11)\n\n");
    for (int k_val = 4; k_val <= 11; k_val++) {
        printf("============================================================\n");
        printf("🔬 Experimento com k = %d\n", k_val);
        printf("============================================================\n");

        if (n_cust < k_val) {
            printf("   ⚠️ Número de clientes (%d) é menor que k (%d). Pulando este k.\n", n_cust, k_val);
            printf("\n");
            continue;
        }


        for (int i = 0; i < n_cust; i++) labels[i] = -1; // Reset labels

        t0 = (double)clock() / CLOCKS_PER_SEC;
        kmeans(X, n_cust, k_val, labels); // Pass k_val
        t1 = (double)clock() / CLOCKS_PER_SEC;
        double t_kmeans = t1 - t0;
        printf("   ✅ KMeans (k=%d) concluído em %.2fs\n", k_val, t_kmeans);

        printf("   ⏱ [CUDA] Calculando Dunn index (GPU)...\n");
        t0 = (double)clock() / CLOCKS_PER_SEC;
        double d_s = dunn_index_cuda(X, labels, n_cust, k_val); // Pass k_val
        t1 = (double)clock() / CLOCKS_PER_SEC;
        double t_s = t1 - t0;
        printf("     → Dunn (CUDA) = %.4f | Tempo: %.2fs\n", d_s, t_s);

                printf("   ⏱ [CUDA] Calculando Dunn index (GPU)...\n");
        t0 = (double)clock() / CLOCKS_PER_SEC;
        t1 = (double)clock() / CLOCKS_PER_SEC;
        double t_cuda = t1 - t0;
        printf("     → Dunn (CUDA) = %.4f | Tempo: %.2fs\n", d_s, t_cuda);

        printf("   🧮 [CPU] Calculando Dunn index (serial)...\n");
        t0 = (double)clock() / CLOCKS_PER_SEC;
        double d_cpu = dunn_index_serial(X, labels, n_cust, k_val);
        t1 = (double)clock() / CLOCKS_PER_SEC;
        double t_cpu = t1 - t0;
        printf("     → Dunn (CPU)  = %.4f | Tempo: %.2fs\n", d_cpu, t_cpu);

        if (t_cuda > 1e-6) {
            printf("     ⚡ Speedup (CPU / CUDA): %.2fx\n", t_cpu / t_cuda);
        } else {
            printf("     ⚠️ CUDA muito rápida para medir speedup com precisão\n");
        }

        printf("\n");
    }

    // Free memory
    for (int i = 0; i < n_cust; i++) { // Use the potentially reduced n_cust from sampling
        if (rfm[i].unique_invoices) { // Check if pointer is not null before freeing
            free(rfm[i].unique_invoices);
        }
    }
    free(rfm);
    free(X);
    if (labels) free(labels); // Check if labels was allocated

    printf("📊 Todos os experimentos concluídos.\n");
    return 0;
}

Overwriting dunn_cuda_fixed.cu


In [ ]:
% nvcc -O2 -o dunn_cuda_fixed dunn_cuda_fixed.cu -lm

In [ ]:
%./dunn_cuda_fixed

📥 Carregando 'cleaned_retail.csv' e calculando RFM...
  → Clientes únicos antes da amostragem: 4129
✅ RFM processado e padronizado (4129 clientes).

🧪 Iniciando experimentos (k = 4..11)

🔬 Experimento com k = 4
   ✅ KMeans (k=4) concluído em 0.00s
   ⏱ [CUDA] Calculando Dunn index (GPU)...
     → Dunn (CUDA) = 0.0000 | Tempo: 0.01s
   ⏱ [CUDA] Calculando Dunn index (GPU)...
     → Dunn (CUDA) = 0.0000 | Tempo: 0.00s
   🧮 [CPU] Calculando Dunn index (serial)...
     → Dunn (CPU)  = 0.0004 | Tempo: 0.02s
     ⚡ Speedup (CPU / CUDA): 23858.00x

🔬 Experimento com k = 5
   ✅ KMeans (k=5) concluído em 0.00s
   ⏱ [CUDA] Calculando Dunn index (GPU)...
     → Dunn (CUDA) = 0.0000 | Tempo: 0.00s
   ⏱ [CUDA] Calculando Dunn index (GPU)...
     → Dunn (CUDA) = 0.0000 | Tempo: 0.00s
   🧮 [CPU] Calculando Dunn index (serial)...
     → Dunn (CPU)  = 0.0002 | Tempo: 0.04s
     ⚡ Speedup (CPU / CUDA): 37290.00x

🔬 Experimento com k = 6
   ✅ KMeans (k=6) concluído em 0.00s
   ⏱ [CUDA] Calculando Dunn in